# Parameter sensitivity study
In this notebook, various parameters are varied, and their impact on the solution is studied.

In [1]:
# Import modules
using Distributions, Random
using LinearAlgebra
using Plots

# Import seismic modules
using seismic.Grids
using seismic.Rays
using seismic.Visualization
using seismic.Inversion

In [2]:
m=1.0e-3;km = 1000m; s=1.0; ms=1e-3s;  # Units

# Case 1: Vary resolution

In [3]:
Random.seed!(123); # Setting the seed

# Define grids
phys_dims = Point(3000km, 2000km)
n_cells_0 = Point(20, 15)

n_ref = 3
grids = Grid[]
for i in 0:n_ref
    n_cells = n_cells_0 * 2^i
    Δx = phys_dims.x / n_cells.x
    Δy = phys_dims.y / n_cells.y
    push!(grids, makeGrid(n_cells.x, n_cells.y, Δx, Δy))
end

In [4]:
# Define source location
sx = 0.0; sy = phys_dims.y / 2
src = Point(sx, sy)

# Distribute floats:
n_recs = round(Int, phys_dims.x * phys_dims.y / (300km)^2)
rx = rand(Uniform(0.1, phys_dims.x - 0.1), n_recs)
ry = rand(Uniform(0.1, phys_dims.y - 0.1), n_recs)
recs = Point{Float64}[]
for i in 1:n_recs
    push!(recs,Point(rx[i], ry[i]))
end

# float-pairs mapping
M = receiverPairs(recs, src, max_degrees = 10, max_distance=1000km);

# Uncertainty parameters
wave_speed = 1.5km/s  # Mean T-wave speed
slowness = 1 / wave_speed

# Prior (solution) slowness variance Rxx, Units: [seconds / meter]
σxx = 2e-4s/km
# spatial decorrelation scale. Units: [m]
λ = 300km   # typical eddy size

# Prior measurement variance Rnn, units: [seconds]
σnn = 10m      # uncertainty in position of resurfacing float
σ_indp = 10ms  # independent TT noise

rnn = Rnn(M*slowness, σnn, σ_indp);  # [s^2]

In [ ]:
# Setup problem and compute uncertainty
P_summary = zeros(n_ref+1)
E_list = Matrix[]
rxx_list = Matrix[]
for (i, grid) in pairs(grids)
    # distance matrix
    D = distanceMatrix(recs, src, grid);
    # relative distance matrix
    E = relativeDistanceMatrix(M,D);
    
    # Prior solution variance
    rxx = Rxx(grid, σxx, λ)  # [(s/m)^2]
    # Uncertainty matrix
    P = uncertaintyMatrix(E, rxx, rnn);
    P_summary[i] = sum(P)
    push!(E_list, E)
    push!(rxx_list, rxx)
end

In [ ]:
cells = n_cells_0.x*n_cells_0.y * [2^i for i in 0:n_ref]
Δxy = [(grid.Δx*grid.Δy)^2 for grid in grids]
N = [(grid.nx*grid.ny)^2 for grid in grids]

rxx_summary = [sum(rxx) for rxx in rxx_list]

In [ ]:
# Uncertainty for varying grid size
P_summary / σxx^2 ./ N
# Does not change, as expected.

# Case 2: Vary prior location uncertainty

In [ ]:
grid = grids[2]

# distance matrix
D = distanceMatrix(recs, src, grid);
# relative distance matrix
E = relativeDistanceMatrix(M,D);

# Prior solution variance
rxx = Rxx(grid, σxx, λ)  # [(s/m)^2]

# Setup problem and compute uncertainty
σnn_arr = [1, 10, 20, 100, 150, 200, 1000, 1e4, 1e5] * m
#σnn_arr = [1, 10, 100] * m
sumP_σnn = Float64[]
for σnn in σnn_arr
    rnn = Rnn(M*slowness, σnn, σ_indp);  # [s^2]
    
    # Uncertainty matrix
    P = uncertaintyMatrix(E, rxx, rnn);
    push!(sumP_σnn, sum(P))
end

In [ ]:
plot(σnn_arr/m, sumP_σnn / σxx^2 / N[2], legend = false, xaxis=:log, ylabel="sum P",xlabel="σnn [m]")
scatter!(σnn_arr/m, sumP_σnn / σxx^2 / N[2])
title!("Sensitivity of uncertainty to changes in σnn")

# Case 3: Independent location noise

In [ ]:
grid = grids[2]

# distance matrix
D = distanceMatrix(recs, src, grid);
# relative distance matrix
E = relativeDistanceMatrix(M,D);

# Prior solution variance
rxx = Rxx(grid, σxx, λ)  # [(s/m)^2]

# Setup problem and compute uncertainty
σ_indp_arr = Array(2:2:20) * ms  # independent TT noise
sumP_σ_indp = Float64[]
for σ_indp in σ_indp_arr
    rnn = Rnn(M*slowness, σnn, σ_indp);  # [s^2]
    
    # Uncertainty matrix
    P = uncertaintyMatrix(E, rxx, rnn);
    push!(sumP_σ_indp, sum(P))
end

In [ ]:
plot(σ_indp_arr/ms, sumP_σ_indp / σxx^2 / N[2], legend = false, ylabel="sum P",xlabel="σi [m]")
scatter!(σ_indp_arr/ms, sumP_σ_indp / σxx^2 / N[2])
title!("Sensitivity of uncertainty to changes in σi")